# Use Amazon Braket to model quantum time evolution

The purpose of this section is to teach you how to model the time evolution of a quantum system described by any Hamiltonian using quantum gates that are provided by most of the QPU’s and simulators offered by Amazon Braket. This section is designed for you to be able to easily replace the Hermitian Hamiltonian that we provide with any Hermitian Hamiltonian of your choosing and to apply these methods to a wide variety of QPU’s and simulators.

## Set up the Hamiltonian

The first step of this tutorial is to set up a random Hermitian Hamiltonian that we are going to use to describe the time evolution of our quantum system. This can be done with the following code:


In [1]:
import numpy as np
#First we define the number of qubits we are going to use.
n = 3
#This defines the Hilbert space of the Hamiltonian. Notice that the size of the Hamiltonian can only be a power of two.
nq = 2**n
#Create a random matrix with values between -1 and 1.
x = np.random.uniform(-1,1, size=(nq,nq))+1j*np.random.uniform(-1,1, size=(nq,nq))
#Use this random matrix to create a Hermitian matrix that we will use for our Hamiltonian.
h = 0.5*x+0.5*x.conj().T

*Note:* We are using a random Hamiltonian just to demonstrate the flexibility that this algorithm has to accommodate any Hermitian Hamiltonian. Feel free to replace this Hamiltonian with another Hermitian Hamiltonian of your choice.

The next step of this tutorial is to decompose this Hamiltonian into a series of Kronecker products of Pauli matrices (Pauli products) so that this problem can be more digestible to a quantum algorithm. One pleasant feature of using Kronecker products of Pauli and identity matrices is that you can decompose any matrix into a sum of these products multiplied by a constant. If we define a three dimensional matrix composed of the three Pauli matrices and a $2×2$ identity matrix as such: $σ~=(I,σ_x,σ_y,σ_z)$, then for our three qubit system, we can define our Hamiltonian as: $H=\frac{1}{N}∑_{i,j,k} h_{i,j,k} \tilde{\sigma}_i⊗\tilde{\sigma}_j⊗\tilde{\sigma}_k$. Here, $N=2n$ where $n$ is the number of qubits, the subscripts on $σ~$ determine the Pauli matrix or identity matrix of interest, and the summation takes into account all possible combinations of Pauli and identity matrices in the Kronecker product. The term $h_{i,j,k}$ is given by the equation: $h_{i,j,k}=TR(\tilde{σ}_i⊗\tilde{σ}_j⊗\tilde{σ}_k H)$. This is implemented with the following code:


In [ ]:
import math
#sigma is a matrix composed of identity and Pauli matrices
sigma = np.zeros((2,2,4), dtype=complex)
sigma[:,:,0] = np.matrix([[1, 0],[0, 1]])
sigma[:,:,1] = np.matrix([[0, 1],[1, 0]])
sigma[:,:,2] = np.matrix([[0, -1j],[1j, 0]])
sigma[:,:,3] = np.matrix([[1, 0],[0, -1]])
#basis will be used to store the Kronecker products of Pauli and identity matrices.
basis = np.zeros((nq,nq,4**n), dtype=complex)
#circind will be used to translate the Kronecker products of Pauli and indentity matrices into quantum gates.
circind = np.zeros((n,4**n), dtype=complex)
#i loops over all possible combinations of Kronecker products
for i in range(4**n):
    matrix = np.zeros((2,2,n), dtype=complex)
    #j loops over all of the qubits
    for j in range(n):
        #num and num2 are constants that determine the combination of Kronecker products of interest.
        num = math.floor(i/(4**j))
        num2 = num % 4
        circind[j,i] = num2
        #matrix is used to store the Pauli and identity matrices used for the Kronecker products of the current iteration of i.
        matrix[:,:,j] = sigma[:,:,num2]
    #matrixsmall and the subsequent iteration of j constructs the Kronecker product relevant for the current iteration of i.
    matrixsmall = matrix[:,:,-1]
    matrixfull = []
    for j in range(n-1):
        del matrixfull
        matrixfull = np.kron(matrix[:,:,-2-j],matrixsmall)
        del matrixsmall
        matrixsmall = matrixfull
    basis[:,:,i] = matrixfull
    del matrixfull
    del matrixsmall

Now that we have obtained all of the Kronecker products with which we are going to decompose our Hamiltonian, it is time to obtain the constants $h_{i,j,k}$:

In [ ]:
#const will be a vector storing all of the constants of interest
const = np.zeros(4**n, dtype=complex)
#null will be a vector that stores where all of the constants in const are zero
null = []
#i loops over all possible combinations of Kronecker products
for i in range(4**n):
    #Below multiplies the Hamiltonian by the Kronecker product of interest.
    gfg = basis[:,:,i] @ h
    #Below takes the trace of this result and stores them in const
    const[i] = gfg.trace()/nq
    if const[i]==0:
        null.append(i)
#Delete components of basis, const, and circind corresponding to where const is zero
if len(null)>0:
    del basis[:,:,null]
    del const[null]
    del circind[:,null]

**Note:** All of the above code pieces work for any value of the number of qubits $n$ and for any appropriately sized Hamiltonian. This code removes all instances where const equals zero (which in this case there are none) as well as the corresponding components of circind in order to save computer time.

The final step to setting up our quantum system is to set up the initial wave function:


In [ ]:
#eigensig is a matrix that stores all of the eigenvectors of the Pauli matrices.
eigensig = np.zeros((2,6), dtype=complex)
eigensig[:,0] = np.array([1, -1])
eigensig[:,1] = np.array([1, 1])
eigensig[:,2] = np.array([1, -1j])
eigensig[:,3] = np.array([1, 1j])
eigensig = (1/math.sqrt(2))*eigensig
eigensig[:,4] = np.array([0, 1])
eigensig[:,5] = np.array([1, 0])
#waveindex determines the three eigenvectors above used to construct the initial state wave function through a Kronecker product.
#The eigenvectors are chosen automatically using a random number generator and the number of these eigenvectors that go into this Kronecker product is equal to the number of qubits.
waveindex = np.random.randint(0,6,n)
#wave stores the wave function of the initial state of the quantum system.
wave = eigensig[:,waveindex[-1]]
#The bottom two lines of code uses a Kronecker product acting on the vectors stored in waveindex to create the initial wave function.
for j in range(n-1):
    wave = np.kron(eigensig[:,waveindex[-2-j]],wave)

**Note:** This code randomly chooses the states that each of the qubits are initialized in; thereby randomly choosing the initial wave function.

### Time evolution and Trotter decomposition

The time evolution of a wave function is normally done with the following equation: $∣Ψ(t)⟩=e−iHt∣Ψ(0)⟩$. Where $∣Ψ(t)⟩$ is the wave function at a time of t and H is the Hamiltonian. However, we have to remember that the decomposition of the Hamiltonian into Kronecker products of Pauli and identity matrices makes it easier to simulate this problem on a quantum computer. We also have to remember that the following inequality exists: $e^{−i(c_1 P_1 + c_2 P_2)t}\neq e^{−i c_1 P_1 t} e^{−i c_2 P_2 t}$, where $P_i$ are the Pauli products that a Hamiltonian can be decomposed into and $c_i$ are the constants that scale these Pauli products appropriately (alternatively referred to as $h_{i,j,k}$). This is unfortunate because this would make calculations much easier for us and is the reason why we use Trotter decomposition. Trotter decomposition is implemented with the following approximation: $e^{−iHt}≈(Π_k e^{−ic_k P_k t/M})^M$. Here, $M$ is the Trotter number and this approximation becomes more accurate as $M$ increases.

## Setting up the quantum circuits

Now that we have decomposed our Hamiltonian, we are going to use these methods to set up a quantum circuit. The good news is that this process involves the implementation of slight variations of a simple circuit numerous times in succession. To start with, we are going to see how the Hamiltonian defined as $H=c_k σ_z ⊗ σ_z ⊗ σ_z$ (with time evolution defined as $e^{−i c_k σ_z ⊗ σ_z ⊗ σ_z t}$) is simulated on a quantum circuit. This quantum circuit is illustrated in the diagram below:
